<a href="https://colab.research.google.com/github/rayvoelker/2015RoeschLibraryInventory/blob/master/misc/voter-data/2018-issue3/CHPL_Issue3_by_voter_precincts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://ballotpedia.org/Hamilton_County,_Ohio,_Issue_3,_Library_Property_Tax_(May_2018)

A property tax to fund library operations and capital improvements was on the ballot for voters in Ohio on May 8, 2018. It was approved.

```
A yes vote was a vote in favor of levying for ten years a property tax at the 
rate of one mill per dollar ($0.10 per $100 of assessed property value) to fund 
library operations and capital improvements.
```

```
A no vote was a vote against levying for ten years a property tax at the rate 
of one mill per dollar ($0.10 per $100 of assessed property value) to fund 
library operations and capital improvements.
```

The tax proposed under Issue 3 was in addition to an existing property tax funding the library at the same rate of one mill per dollar. The combined property taxes would bring in approximately $34 million for the lirbary per year, according to Cincinnati.com (https://ballotpedia.org/Hamilton_County,_Ohio,_Issue_3,_Library_Property_Tax_(May_2018)#cite_note-Mill-1)

---

Using Voter Data to compare to CHPL Patron data

1. Hamiltion County Ohio Board of Elections -- Voter lists:
   - https://votehamiltoncountyohio.gov/campaign-media/voter-lists/

hamilton-county-voter-precincts:

https://data-cagisportal.opendata.arcgis.com/datasets/hamilton-county-voter-precincts/explore



In [2]:
!pip install -U duckdb --quiet

import duckdb
import pandas as pd
import altair as alt

from google.colab import data_table
# enable the formatter for more functionality with tabular data
data_table.enable_dataframe_formatter()

     |████████████████████████████████| 15.7 MB 3.8 MB/s 


In [4]:
# get the current Hamilton County registerd voter list ...
# downloaded and compressed from here: https://votehamiltoncountyohio.gov/campaign-media/voter-lists/
!wget --quiet https://github.com/cincinnatilibrary/collection-analysis/raw/0b2571dacfc8ff917ad5bd16b95faa136e6c98b0/misc/voter-data/2018-issue3/VoterListExport-20220627.tar.xz --output-document=VoterListExport.tar.xz

# get the results of the election ...
!wget --quiet https://github.com/cincinnatilibrary/collection-analysis/raw/1768a4a877ba5dc802327c34bb80e0af73ee80b7/misc/voter-data/2018-issue3/2018_hamilton_county_issue_3_results.xlsx

# get the shape files related to the Hamilton County Voter Precincts
!wget --quiet https://github.com/cincinnatilibrary/collection-analysis/raw/1768a4a877ba5dc802327c34bb80e0af73ee80b7/misc/voter-data/2018-issue3/Hamilton_County_Voter_Precincts.zip

# load it into a dataframe ... 
voter_df = pd.read_csv(
    "VoterListExport.tar.xz", 
    parse_dates=['RegisteredDate'],
    low_memory=False
)

In [5]:
# rename the column to "VoterID"
voter_df = voter_df.rename(columns={'VoterListExport-20220627-no.csv': 'VoterID'})

# convert the VoterID to integer (it defaulted to numeric since there maybe a null)
voter_df['VoterID'] = voter_df['VoterID'].astype("Int64")

# PrecinctNum should also be an integer value
voter_df['PrecinctNum'] = voter_df['PrecinctNum'].astype("Int64")

In [6]:
# DuckDB allows us to efficently run SQL on the Pandas DataFrame!

# connect to an in-memory database
con = duckdb.connect()

In [7]:
# 
result_df = con.execute("""\
  SELECT
    VoterId,
    AddressNumber,
    AddressStreet,
    AddressSuffix,
    CityName,
    AddressZip,
    BirthYear,
    PartyCode,
    PrecinctNum,
    PrecinctName,
    RegisteredDate,
    strftime('%Y', RegisteredDate) as reg_year, 
    Status, 
    PRIMARY_MAY_2022
  FROM
    voter_df
  ORDER BY RANDOM()
  LIMIT 1000
  """
).df()

result_df

,VoterID,AddressNumber,AddressStreet,AddressSuffix,CityName,AddressZip,BirthYear,PartyCode,PrecinctNum,PrecinctName,RegisteredDate,reg_year,Status,PRIMARY_MAY_2022
0,1987122,1457.0,HOFFNER,ST,CINCINNATI,45231.0,1987.0,NON,4121,MT HEALTHY 2-A,2018-09-18,2018,A,NaN
1,1709041,749.0,CEDAR POINT,DR,CINCINNATI,45230.0,1973.0,REP,5608,ANDERSON H,2010-05-04,2010,A,P-REP
2,1403063,155.0,KINSEY,AVE,CINCINNATI,45219.0,1979.0,NON,801,CINCINNATI 8-A,2001-10-10,2001,A,NaN
3,1993267,2614.0,RIDGECLIFF,AVE,CINCINNATI,45212.0,1978.0,NON,1409,CINCINNATI 14-I,2018-10-08,2018,A,NaN
4,2068636,6004.0,BENDER,RD,CINCINNATI,45233.0,1977.0,NON,6518,DELHI R,2020-10-05,2020,A,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1662415,1661.0,CEDAR,AVE,CINCINNATI,45224.0,1963.0,NON,2314,CINCINNATI 23-N,2008-09-04,2008,A,NaN
996,1933266,6464.0,TEAKWOOD,CT,CINCINNATI,45224.0,1996.0,DEM,2315,CINCINNATI 23-O,2016-10-07,2016,A,NaN
997,628567,3432.0,WELLSTON,PL,CINCINNATI,45208.0,1948.0,NON,407,CINCINNATI 4-G,1989-07-17,1989,A,NaN
998,1973025,3960.0,GLENMORE,AVE,CINCINNATI,45211.0,1957.0,REP,2921,CHEVIOT 2-A,2018-05-15,2018,A,P-REP


In [8]:
result_df = con.execute("""\
  SELECT
    cast(PrecinctNum as INTEGER) as PrecinctNum,
    PrecinctName,
    count(DISTINCT VoterId)
  FROM
    voter_df
  WHERE
    PrecinctNum is not null
  GROUP BY
    1,2
  ORDER BY
    1 ASC
  """
).df()

result_df

,PrecinctNum,PrecinctName,"count(DISTINCT ""VoterId"")"
0,101,CINCINNATI 1-A,314
1,102,CINCINNATI 1-B,1293
2,103,CINCINNATI 1-C,1204
3,104,CINCINNATI 1-D,791
4,105,CINCINNATI 1-E,1303
...,...,...,...
557,9501,WHITEWATER A,1329
558,9502,WHITEWATER B,1490
559,9503,WHITEWATER C,1189
560,9701,WOODLAWN A,1066


In [9]:
!pip install -U geopandas --quiet
!pip install -U folium matplotlib mapclassify --quiet

     |████████████████████████████████| 1.0 MB 4.4 MB/s 
     |████████████████████████████████| 6.3 MB 36.7 MB/s 
     |████████████████████████████████| 16.7 MB 428 kB/s 
     |████████████████████████████████| 95 kB 2.5 MB/s 
     |████████████████████████████████| 11.2 MB 24.1 MB/s 
     |████████████████████████████████| 944 kB 50.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.12.1.post1 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [10]:
# # enable a driver for reading KML file formats
# gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
# print(fiona.supported_drivers)

# import pandas as pd
# import geopandas as gpd
# import fiona
# import duckdb

# # connect to an in-memory database
# con = duckdb.connect()

# gdf_org = gpd.read_file(
#     "Hamilton_County_Voter_Precincts.kml",
#      driver="KML"
# )

# gdf_org.explore()

In [11]:
import pandas as pd
import geopandas as gpd
import fiona
import duckdb

# connect to an in-memory database
con = duckdb.connect()

gdf_org = gpd.read_file(
    "/content/Hamilton_County_Voter_Precincts.zip",
)

In [12]:
# the results data by precinct
issue3_df = pd.read_excel(
    "2018_hamilton_county_issue_3_results.xlsx"
)

In [29]:
# find the highest percentage of "against" by precinct
# ... just return the top half
result_df = con.execute("""\
  with voter_data as (
    SELECT
      "PRC #" as PCT, -- keeping it how it is called in the shape data
      "PRECINCT" as precinct_name,
      "REGISTERED VOTERS TOTAL" as registered_total,
      "BALLOTS CAST TOTAL" as cast_total,
      "FOR" as for_votes,
      "AGAINST" as against_votes
    FROM
      issue3_df
  )
  SELECT
    v.*,
    round((for_votes / (cast_total * 1.0) ) * 100.0, 2) as percent_for,
    round((against_votes / (cast_total * 1.0) ) * 100.0, 2) as percent_against
  FROM
    voter_data as v
  ORDER BY
    percent_against DESC
  LIMIT 
    -- limit to half the precincts ... 
    (SELECT count(*) / 2 from voter_data)
  """
).df()

result_df

,PCT,precinct_name,registered_total,cast_total,for_votes,against_votes,percent_for,percent_against
0,6209,6209 COLUM I,11,2,0,2,0.00,100.00
1,6210,6210 COLUM J,2,2,0,2,0.00,100.00
2,5909,5909 COLE I,919,228,66,160,28.95,70.18
3,7602,7602 HARR BB,1244,276,88,185,31.88,67.03
4,5906,5906 COLE F,1243,286,93,189,32.52,66.08
...,...,...,...,...,...,...,...,...
274,708,0708 CIN 7-H,1148,147,88,53,59.86,36.05
275,6501,6501 DELHI A,1173,139,88,50,63.31,35.97
276,3212,3212 FR PK L,1095,181,114,65,62.98,35.91
277,3141,3141 DR PK 4-A,1105,201,121,72,60.20,35.82


In [35]:
# find the precincts where more than half "against" 
#
result_df = con.execute("""\
  with voter_data as (
    SELECT
      "PRC #" as PCT, -- keeping it how it is called in the shape data
      "PRECINCT" as precinct_name,
      "REGISTERED VOTERS TOTAL" as registered_total,
      "BALLOTS CAST TOTAL" as cast_total,
      "FOR" as for_votes,
      "AGAINST" as against_votes
    FROM
      issue3_df
  )
  SELECT
    v.*,
    round((for_votes / (cast_total * 1.0) ) * 100.0, 2) as percent_for,
    round((against_votes / (cast_total * 1.0) ) * 100.0, 2) as percent_against
  FROM
    voter_data as v
  WHERE
    round((against_votes / (cast_total * 1.0) ) * 100.0, 2) > 50.0
  ORDER BY
    percent_against DESC
  
  """
).df()

result_df

,PCT,precinct_name,registered_total,cast_total,for_votes,against_votes,percent_for,percent_against
0,6209,6209 COLUM I,11,2,0,2,0.00,100.00
1,6210,6210 COLUM J,2,2,0,2,0.00,100.00
2,5909,5909 COLE I,919,228,66,160,28.95,70.18
3,7602,7602 HARR BB,1244,276,88,185,31.88,67.03
4,5906,5906 COLE F,1243,286,93,189,32.52,66.08
5,5901,5901 COLE A,1173,339,112,220,33.04,64.90
6,6401,6401 CROSB A,968,119,41,77,34.45,64.71
7,9503,9503 WHWTR C,684,63,22,40,34.92,63.49
8,6402,6402 CROSB B,1223,294,108,185,36.73,62.93
9,7339,7339 GREEN MM,664,162,60,101,37.04,62.35


In [36]:
gdf_org.columns

Index(['OBJECTID', 'PRECINCT', 'NEWADDR', 'PCT', 'PRC_NAME', 'LOCATION',
       'ADDRESS', 'CITY', 'ZIP', 'GLOBALID', 'SE_ANNO_CA', 'GLOBALID_1',
       'ENTRANCE', 'SHAPEAREA', 'SHAPELEN', 'geometry'],
      dtype='object')

In [37]:
result_df.columns

Index(['PCT', 'precinct_name', 'registered_total', 'cast_total', 'for_votes',
       'against_votes', 'percent_for', 'percent_against'],
      dtype='object')

In [40]:
# merge the above dataframe and the geodataframe

gdf = gdf_org.merge(result_df, on='PCT', how='inner')

gdf.explore(
    "against_votes",
    cmap="YlOrRd"
)

In [42]:
# export the above data as KML data for use in Gale Analytics ...
import fiona
fiona.supported_drivers['KML'] = 'rw'

gdf.to_file('service_area.kml', driver='KML')